# Units and Labels

prysm tracks units using [astropy.units](https://docs.astropy.org/en/stable/units/), but purposfully avoids `Quantity` objects for arrays, since these are hard-bound to `numpy`, and prysm's backend system allows api-compatible replacements such as [cupy](https://docs-cupy.chainer.org/en/stable/) to be used.  The label system used by prysm allows flexibility override of its default choices for axis labels when [plotting](./plotting.html).

We begin by performing some imports:

In [ ]:
%load_ext autoreload
%autoreload 2

from prysm import Units, Labels, NollZernike, config
from prysm import wavelengths as wvl

from astropy import units as u

from matplotlib import pyplot as plt
%matplotlib inline

`Units` instances are very simple to create and have just four parameters, `x`, `z`, `y`, and `wavelength`.  The order of `y` and `z` is swapped, since you rarely want y and x to have different units.  `y=None` (the default) will copy the x unit to y.  We can put this into action easily when we do not care about wavelengths (e.g. for images):

In [ ]:
unit_pack = Units(x=u.mm, z=u.nm)

To include a wavelength, we need a unit that describes one wavelength of light.  To facilitate this, prysm includes a `mkwvl` function to make wavelengths of arbitrary size as well as a `wavelengths` sub-module populated with common laser wavelengths:

In [ ]:
print('built-in wavelengths are:')
print(wvl.__all__)

telecon_wvl = wvl.mkwvl(1550, u.nm) # == mkwvl(1.55)

telecom_unit_pack = Units(u.mm, u.m, wavelength=telecon_wvl) # note meters for z (phase)
print('telecon unit pack\t', telecom_unit_pack)

these unit packs are threaded into prysm's object system with the `units` keyword argument:

In [ ]:
z = NollZernike(Z25=1e-6, units=telecom_unit_pack)
z.plot2d()  # units visible on plot

The wavelength can be checked via its `represents` property if you lose track of what it is:

In [ ]:
z.units.wavelength.represents

Turning our attention to the labels on this plot, those too can be adjusted by creating a `Labels` object and passing it with the `labels` keyword argument:

In [ ]:
label_pack = Labels(xy_base='Optical Table', z='Height',
                    xy_additions=['exx', 'why'], xy_addition_side='left',
                    addition_joiner=' space ', unit_prefix='>', unit_suffix='<',
                    unit_joiner=' spaaaaace ')

z = NollZernike(units=unit_pack, labels=label_pack)
z.plot2d()

Most of these have sensible defaults, a more reasonable usage would be:

In [ ]:
label_pack = Labels(xy_base='Optical Table', z='Height')
z = NollZernike(units=unit_pack, labels=label_pack)
z.plot2d()

Finally, across the library the default units and labels for the various classes can be controlled with the config class:

In [ ]:
config_items = vars(config)
for key in config_items:
    if 'units' in key:
        print(key, '\t\t', config_items[key])
    elif 'labels' in key:
        print(key, '\t\t', 'X/Y', config_items[key].xy_base, 'Z', config_items[key]._z)